# Étape 3 : Entraînement et Test de la Cascade de Traduction

Ce notebook permet de lancer l'entraînement ASR (Speech-to-Text) sur CPU et de tester la chaîne de traduction finale (Mina ➔ Éwé ➔ Français).

In [1]:
import sys
import importlib
from pathlib import Path
if str(Path.cwd().parent) not in sys.path:
    sys.path.append(str(Path.cwd().parent))

import pandas as pd
import torch
import torchaudio
from transformers import WhisperForConditionalGeneration, WhisperProcessor, AutoTokenizer, AutoModelForSeq2SeqLM, MarianTokenizer, MarianMTModel
from src.config import settings
import src.models.train_whisper_cpu

# Reload de nos modules persos
import src.models.translation_mina_ewe
import src.models.translation_ewe_fr
import src.pipeline.translate_cascade
importlib.reload(src.models.train_whisper_cpu)
importlib.reload(src.models.translation_mina_ewe)
importlib.reload(src.models.translation_ewe_fr)
importlib.reload(src.pipeline.translate_cascade)

from src.models.train_whisper_cpu import train_whisper_on_cpu
from src.pipeline.translate_cascade import TranslationCascade

c:\EPL\MTH2321\MTH_2321_APEKE\Projet_traduction_ewe_francais\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Entraînement ASR (Whisper sur CPU)

 (> **Note :** L'entraînement sur CPU est lent. Nous utilisons un gel des couches (freeze) à 90% pour accélérer le processus. Les hyperparamètres sont configurés dans `src/config/settings.py`.

In [2]:
# Chargement du dataset préparé à l'étape 2
dataset_path = settings.PROCESSED_DIR / "bible_asr_dataset.csv"

if dataset_path.exists():
    df = pd.read_csv(dataset_path)
    print(f"Dataset chargé : {len(df)} exemples.")
    
    # Lancement de l'initialisation de l'entraînement
    # Note: assurez-vous d'avoir assez de RAM
    model, processor = train_whisper_on_cpu(None) 
    
    # --- SAUVEGARDE DU MODÈLE ENTRAÎNÉ (EXPORT) ---
    export_dir = settings.PROJECT_ROOT / "models" / "whisper-ewe-mina-final"
    print(f"Exportation du modèle ASR vers {export_dir}...")
    model.save_pretrained(export_dir)
    processor.save_pretrained(export_dir)
    print("Modèle ASR exporté avec succès !")
else:
    print("Erreur : Le dataset ASR n'a pas encore été généré. Veuillez exécuter le notebook 02 d'abord.")

Dataset chargé : 801 exemples.
--- Initialisation Entraînement Whisper (openai/whisper-base) ---
Chargement du dataset depuis C:\EPL\MTH2321\MTH_2321_APEKE\Projet_traduction_ewe_francais\data\processed\bible_asr_dataset.csv...


Generating train split: 801 examples [00:00, 5634.39 examples/s]


Préparation du dataset (Audio -> Mel Spectrogram + Tokenization)...
Dataset keys before map: dict_keys(['train'])


Map: 100%|██████████| 801/801 [01:31<00:00,  8.79 examples/s]
c:\EPL\MTH2321\MTH_2321_APEKE\Projet_traduction_ewe_francais\venv\Lib\site-packages\transformers\training_args.py:1636: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
c:\EPL\MTH2321\MTH_2321_APEKE\Projet_traduction_ewe_francais\src\models\train_whisper_cpu.py:158: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Dataset type after map: <class 'datasets.dataset_dict.DatasetDict'>
Dataset keys after map: dict_keys(['train'])
Modèle de 72593920 params. Gel de 90.0% pour CPU...
ATTENTION: 'test' split manquant. Keys actuels : dict_keys(['train'])
Tentative de split de secours...
Mise à jour des keys : dict_keys(['train', 'test'])
Démarrage de l'entraînement...


You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
10,4.797400,4.315985
20,4.099200,3.867604


c:\EPL\MTH2321\MTH_2321_APEKE\Projet_traduction_ewe_francais\venv\Lib\site-packages\transformers\modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


Entraînement terminé.
Exportation du modèle ASR vers C:\EPL\MTH2321\MTH_2321_APEKE\Projet_traduction_ewe_francais\models\whisper-ewe-mina-final...
Modèle ASR exporté avec succès !


## 2. Exportation des autres modèles (NLLB et Opus-MT)

Pour avoir une solution 100% autonome, nous téléchargeons et sauvegardons localement les modèles de traduction.

In [3]:
print("Téléchargement et export des modèles de traduction...")

# 1. Export NLLB (Mina -> Ewe)
nllb_model_name = "facebook/nllb-200-distilled-600M"
nllb_path = settings.PROJECT_ROOT / "models" / "nllb-mina-ewe-local"

if not nllb_path.exists():
    print(f"Téléchargement de {nllb_model_name}...")
    tokenizer = AutoTokenizer.from_pretrained(nllb_model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(nllb_model_name)
    
    print(f"Sauvegarde dans {nllb_path}...")
    model.save_pretrained(nllb_path)
    tokenizer.save_pretrained(nllb_path)
else:
    print(f"Modèle NLLB déjà présent dans {nllb_path}")

# 2. Export Opus-MT (Ewe -> Français)
opus_model_name = "Helsinki-NLP/opus-mt-ee-fr"
opus_path = settings.PROJECT_ROOT / "models" / "opus-ewe-fr-local"

if not opus_path.exists():
    print(f"Téléchargement de {opus_model_name}...")
    tokenizer = MarianTokenizer.from_pretrained(opus_model_name)
    model = MarianMTModel.from_pretrained(opus_model_name)
    
    print(f"Sauvegarde dans {opus_path}...")
    model.save_pretrained(opus_path)
    tokenizer.save_pretrained(opus_path)
else:
    print(f"Modèle Opus déjà présent dans {opus_path}")

print("Tous les modèles sont prêts localement !")

Téléchargement et export des modèles de traduction...
Modèle NLLB déjà présent dans C:\EPL\MTH2321\MTH_2321_APEKE\Projet_traduction_ewe_francais\models\nllb-mina-ewe-local
Modèle Opus déjà présent dans C:\EPL\MTH2321\MTH_2321_APEKE\Projet_traduction_ewe_francais\models\opus-ewe-fr-local
Tous les modèles sont prêts localement !


## 3. Test Audio Complet avec les 3 Modèles Exportés (Local)

Nous chargeons maintenant **les 3 modèles** uniquement depuis le dossier `models/`.

In [6]:
# Chemins locaux
asr_path = settings.PROJECT_ROOT / "models" / "whisper-ewe-mina-final"
nllb_path = settings.PROJECT_ROOT / "models" / "nllb-mina-ewe-local"
opus_path = settings.PROJECT_ROOT / "models" / "opus-ewe-fr-local"

print("1. Chargement du modèle ASR local...")
try:
    loaded_processor = WhisperProcessor.from_pretrained(asr_path)
    loaded_model = WhisperForConditionalGeneration.from_pretrained(asr_path)
    print("   OK.")
except Exception as e:
    print(f"   Erreur ASR (il faut lancer l'entraînement avant) : {e}")
    # Fallback pour ne pas bloquer si pas entraîné dans cette session
    loaded_processor = processor 
    loaded_model = model

print("2. Initialisation de la Cascade avec les modèles locaux...")
cascade = TranslationCascade(nllb_path=str(nllb_path), opus_path=str(opus_path))
print("   OK.")


1. Chargement du modèle ASR local...


INFO:src.pipeline.translate_cascade:Initialisation de la cascade de traduction...
INFO:src.models.translation_mina_ewe:Chargement du modèle Mina-Ewe : C:\EPL\MTH2321\MTH_2321_APEKE\Projet_traduction_ewe_francais\models\nllb-mina-ewe-local


   OK.
2. Initialisation de la Cascade avec les modèles locaux...


The tokenizer you are loading from 'C:\EPL\MTH2321\MTH_2321_APEKE\Projet_traduction_ewe_francais\models\nllb-mina-ewe-local' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
INFO:src.models.translation_ewe_fr:Chargement du modèle Transformers C:\EPL\MTH2321\MTH_2321_APEKE\Projet_traduction_ewe_francais\models\opus-ewe-fr-local


   OK.


In [11]:
# --- INFERENCE ---
import soundfile as sf
import scipy.signal
import numpy as np
import torch

sample_audio = settings.PROCESSED_DIR / "audio_16k" / "gegbe_gen_01.wav"

if sample_audio.exists():
    print(f"\nTraitement audio : {sample_audio.name}")
    
    waveform, sr = sf.read(sample_audio)
    
    print(f"Durée originale : {len(waveform) / sr:.2f} secondes")
    print(f"Échantillons    : {len(waveform):,}")
    print(f"Sample rate lu  : {sr} Hz")
    
    # Mono si stéréo
    if len(waveform.shape) > 1:
        waveform = waveform.mean(axis=1)
        
    # Resample si besoin
    if sr != 16000:
        num_samples = int(round(len(waveform) * 16000 / sr))
        waveform = scipy.signal.resample(waveform, num_samples)
    
    # Préparation Whisper
    input_features = loaded_processor(
        waveform,
        sampling_rate=16000,
        return_tensors="pt"
    ).input_features
    
    # Très important : déplacer sur le bon device (GPU si disponible)
    device = next(loaded_model.parameters()).device
    input_features = input_features.to(device)
    
    with torch.no_grad():
        predicted_ids = loaded_model.generate(
            input_features,
            language=None,                     # ou "mina" / None si détection auto
            task="transcribe",
            return_timestamps=True,             # active le découpage automatique 30s + stitching
            condition_on_prev_tokens=True,
            temperature=[0.0, 0.2, 0.4, 0.6, 0.8, 1.0],  # fallback si répétitions/hallucinations
            compression_ratio_threshold=1.35,
            logprob_threshold=-1.0,
            no_speech_threshold=0.6,
        )
    
    transcription = loaded_processor.batch_decode(
        predicted_ids,
        skip_special_tokens=True
    )[0]
    
    print(f"\nTranscription (ASR) :\n{transcription}\n")
    
    final_result = cascade.translate_mina_to_french(transcription)
    
    print("=== RÉSULTAT 100% LOCAL ===")
    print(f"TEXTE (ASR) : {final_result['mina']}")
    print(f"PIVOT (EWE) : {final_result['ewe']}")
    print(f"TRAD (FR)   : {final_result['french']}")

else:
    print("Fichier audio non trouvé.")


Traitement audio : gegbe_gen_01.wav
Durée originale : 368.40 secondes
Échantillons    : 5,894,400
Sample rate lu  : 16000 Hz


INFO:src.pipeline.translate_cascade:Source (Mina):  gone le Jébeho Man ɛtɔɛtɔtɔ 2gbansihi habe dɔro dɔ su hone jéjai Maodó dɔ jynkuti ku annibã ayiimala na manh e be le ge bello kyi hì wì tì tìrìpɔ la gone jɔ wì dìbe apū bàjɔ à vòmaɔ hè bŋ gbãn dìna sa le seò zìf dìwo mɔ gŋ manbe kèngɔ nè vèemé K



Transcription (ASR) :
 gone le Jébeho Man ɛtɔɛtɔtɔ 2gbansihi habe dɔro dɔ su hone jéjai Maodó dɔ jynkuti ku annibã ayiimala na manh e be le ge bello kyi hì wì tì tìrìpɔ la gone jɔ wì dìbe apū bàjɔ à vòmaɔ hè bŋ gbãn dìna sa le seò zìf dìwo mɔ gŋ manbe kèngɔ nè vèemé K



INFO:src.pipeline.translate_cascade:Pivot (Ewe): dzo le Yebeho Man ɛtɔtɔtɔtɔ 2gbansihi habe dɔro dɔ su hone jéjai Maodó dɔ jynkuti ku annibã ayiimala na man e be le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le
INFO:src.pipeline.translate_cascade:Cible (Français): À Yeho Mansisihi, dans la ville de Madjalóldoa, on trouve auprès d'Adyjalóo, où se trouve manitia, dans un quartier situé à proximité de chez elle, où se trouve une maladie grave.


=== RÉSULTAT 100% LOCAL ===
TEXTE (ASR) :  gone le Jébeho Man ɛtɔɛtɔtɔ 2gbansihi habe dɔro dɔ su hone jéjai Maodó dɔ jynkuti ku annibã ayiimala na manh e be le ge bello kyi hì wì tì tìrìpɔ la gone jɔ wì dìbe apū bàjɔ à vòmaɔ hè bŋ gbãn dìna sa le seò zìf dìwo mɔ gŋ manbe kèngɔ nè vèemé K
PIVOT (EWE) : dzo le Yebeho Man ɛtɔtɔtɔtɔ 2gbansihi habe dɔro dɔ su hone jéjai Maodó dɔ jynkuti ku annibã ayiimala na man e be le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le le
TRAD (FR)   : À Yeho Mansisihi, dans la ville de Madjalóldoa, on trouve auprès d'Adyjalóo, où se trouve manitia, dans un quartier situé à proximité de chez elle, où se trouve une maladie grave.
